## Predicting the number of goals a player will score in a season, based on previous season performance

Output stats

1- Goals 

Input stats

2- Prev. year goals 

3- prev. year xG 

4- prev. year team goals

5- prev. year team xG 

6- Player age.

7- Player position.

8 - minutes played

9 - did they change club?

10 - did they change league (if so, which) can we rank the leagues in some way


In [2]:
import numpy as np 
import pandas as pd 
import plotly
from plotly import graph_objects as go
import os 


## Loading the data

In [3]:
#create blank data_frame with necessary columns
column_names = pd.read_csv('Project_data/player_stats/Bundesliga_2014').columns
df = pd.DataFrame(columns=column_names)
#get all the files in the folder and extract the league and year from them
for file_suffix in os.listdir('Project_data/player_stats'):
    len_of_file = len(file_suffix)
    year = file_suffix[-4:]
    league = file_suffix[:len_of_file-5]
    df_temp = pd.read_csv(f'Project_data/player_stats/{file_suffix}')
    df_temp['league'] = league
    df_temp['year'] = year
    df=df.append(df_temp)
df = df.reset_index().drop(columns='Unnamed: 0')
df = df.drop(columns='index')

## Data cleaning and combining

We need to append "Last Year stats" for each player

In [4]:
#create a dictionary to reference that shows each of the 'previous year values
last_year_dictionary = {
    '2015':'2014',
    '2016':'2015',
    '2017':'2016',
    '2018':'2017',
    '2019': '2018',
    '2020': '2019'
}
#Create blank rows
df[['goals_LY','npg_LY','xG_LY','npxG_LY','time_LY']] = ""


In [5]:
df.columns

Index(['id', 'player_name', 'games', 'time', 'goals', 'xG', 'assists', 'xA',
       'shots', 'key_passes', 'yellow_cards', 'red_cards', 'position',
       'team_title', 'npg', 'npxG', 'xGChain', 'xGBuildup', 'league', 'year',
       'goals_LY', 'npg_LY', 'xG_LY', 'npxG_LY', 'time_LY'],
      dtype='object')

In [6]:
#iterate over the length of the df, and add the previous years metrics.
for i in range(len(df)):
    if df.iloc[i]['year'] != '2014':
        year = df.loc[i]['year']
        id = df.loc[i]['id']
        try:
            df.iloc[i,20] = np.float(df.loc[(df['id'] == id) & (df['year'] == last_year_dictionary[year])]['goals'])
            df.iloc[i,21] = np.float(df.loc[(df['id'] == id) & (df['year'] == last_year_dictionary[year])]['npg'])
            df.iloc[i,22] = np.float(df.loc[(df['id'] == id) & (df['year'] == last_year_dictionary[year])]['xG'])
            df.iloc[i,23] = np.float(df.loc[(df['id'] == id) & (df['year'] == last_year_dictionary[year])]['npxG'])
            df.iloc[i,24] = np.float(df.loc[(df['id'] == id) & (df['year'] == last_year_dictionary[year])]['time'])
        except:
            continue
        

In [7]:
df.describe()

,xG,xA,npxG,xGChain,xGBuildup
count,18506.000000,18506.000000,18506.000000,18506.000000,18506.000000
mean,1.830020,1.282332,1.670554,4.816423,2.831072
std,3.123780,1.807406,2.741822,5.195113,3.068444
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.080369,0.060809,0.079915,0.974663,0.545860
50%,0.667634,0.576776,0.649842,3.265941,1.938602
75%,2.140849,1.790841,2.014398,6.935059,4.071566
max,39.308761,20.620707,32.117727,54.753610,28.058852


In [8]:
#Make sure we have all the new columns as floats
df["goals_LY"] = pd.to_numeric(df["goals_LY"], downcast="float")
df["npg_LY"] = pd.to_numeric(df["npg_LY"], downcast="float")
df["xG_LY"] = pd.to_numeric(df["xG_LY"], downcast="float")
df["npxG_LY"] = pd.to_numeric(df["npxG_LY"], downcast="float")
df["time_LY"] = pd.to_numeric(df["time_LY"], downcast="float")

## Data Normalisation

We have a couple of small issues.

1) The French league finished prematurely in 2019 (28 games)

2) The German league only has 34 games, compared to 38 for the other season.

To fix this, we will normalise all seasons to a 38 game season. This will help us if we have players transferring from the German league, too.

In [9]:
#Create a function which creates a normalising constant for each league
def multiplier_code(league):
    if league.startswith('Bundesliga'):
        return 38/34
    elif league == 'Ligue 1_2019':
        return 38/28
    else:
        return 1
#Create a league_year column
df['league_year'] = df['league'] + '_' + df['year']
#call the function on league_year and produce a column with norm constant
df['normalise_constant'] = df['league_year'].apply(multiplier_code)


In [10]:
normalisable_columns = ['goals','time','xG','assists','xA','shots','key_passes','yellow_cards', 'red_cards','npg', 'npxG', 'xGChain', 'xGBuildup','goals_LY', 'npg_LY', 'xG_LY', 'npxG_LY','time_LY']
#now multiply the values in the relevant columns by the normalising constant for that league.
for col in normalisable_columns:
    df[f'{col}_norm'] = df[col] * df['normalise_constant']

## Initial Data visualisation

In [11]:
from plotly import subplots

In [12]:
#Histogram of goals per season for each different league.
fig = plotly.subplots.make_subplots(rows=1, cols=2)
bins = 20
hist_1 = go.Histogram(x=df[df['league'].str.startswith('EPL')]['goals_norm'], name = 'EPL',nbinsx=bins)
hist_2 = go.Histogram(x=df[df['league'].str.startswith('Bundesliga')]['goals_norm'], name = 'Bundesliga',nbinsx=bins)
hist_3 = go.Histogram(x=df[df['league'].str.startswith('Ligue')]['goals_norm'], name = 'Ligue 1',nbinsx=bins)
hist_4 = go.Histogram(x=df[df['league'].str.startswith('Serie')]['goals_norm'], name = 'Serie A',nbinsx=bins)
hist_5 = go.Histogram(x=df[df['league'].str.startswith('La')]['goals_norm'], name = 'La Liga',nbinsx=bins)
fig.add_trace(hist_1,row=1,col=2)
fig.add_trace(hist_2,row=1,col=2)
fig.add_trace(hist_3,row=1,col=2)
fig.add_trace(hist_4,row=1,col=2)
fig.add_trace(hist_5,row=1,col=2)
fig.add_trace(hist_1,row=1,col=1)
fig.add_trace(hist_2,row=1,col=1)
fig.add_trace(hist_3,row=1,col=1)
fig.add_trace(hist_4,row=1,col=1)
fig.add_trace(hist_5,row=1,col=1)
fig['layout']['yaxis2'].update(type='log',title='Log scale of number of players')
fig['layout']['yaxis1'].update(title='Number of players')
fig.update_xaxes(title = 'Normalised goals in a season')
fig.update_layout(width=1000,height=500,title='Linear and Log Scale plots of goals per season by individual players',title_x=0.5)
fig.show()


## Using Chi Square to assess impact of League

It looks like the league doesn't have a huge impact on the figures. We can use a Chi-Squared analysis to analyse if we can remove 'league' from the input variables

In [13]:
#Create the total goals for every league
leagues = ['EPL','Bund','Ligue','Serie','La_liga']
league_goals_player = pd.DataFrame(columns = ['League','Goals','Players'])
for league in leagues:
    #Number of goals in the league
    globals()[f'{league}_goals'] = int(np.sum(df[df['league'].str.startswith(league)]['goals_norm']))
    #Number of players
    globals()[f'{league}_players'] = len(df[df['league'].str.startswith(league)])
    #append stats to dataframe
    league_goals_player = league_goals_player.append({'League':league,'Goals':globals()[f'{league}_goals'],'Players':globals()[f'{league}_players']},ignore_index=True)

In [14]:
#with chisquared, you compare the number of goals to the expected number of goals based on the size of the population
Total_goals_div_players = (np.sum(league_goals_player['Goals']) / np.sum(league_goals_player['Players']))
league_goals_player['Exp_goals'] = (Total_goals_div_players * league_goals_player['Players']).astype(int)
display(league_goals_player)

,League,Goals,Players,Exp_goals
0,EPL,6824,3648,6707
1,Bund,6776,3302,6071
2,Ligue,6571,3850,7079
3,Serie,7099,3896,7163
4,La_liga,6758,3810,7005


First let's do a manual Chi-Square, and then compare to the built in function from scipy

In [15]:
#Then, calculate goals - expected goals, and divide by expected goals, and compare to chi squared test based on 4 dof
SSE = 0
for i in range(5):
    goals = league_goals_player.loc[i]['Goals']
    exp_goals = league_goals_player.loc[i]['Exp_goals']
    SSE += (goals-exp_goals)**2/exp_goals
print(f'Chi-Squared score is: {SSE}')

Chi-Squared score is: 129.6457665003485


This means that the league makes a significant difference to the mean goals per player, as the required cs value to have a greater than 99.5% confidence of rejecting the null is only 0.207

In [16]:
from scipy.stats import chisquare
display(chisquare(f_obs=league_goals_player['Goals'],f_exp= league_goals_player['Exp_goals']))

Power_divergenceResult(statistic=129.6457665003485, pvalue=4.636138538825115e-27)

## Excellent! Same result :-)

## Z Test to assess if changing club or league effects goals

One possible data point to include is whether someone changes club. Let's use an F test to analyse whether changing club or league has a significant effect on your "Change in goals v last year"

In [17]:
'''First create goals delta, and then a column for the team you were playing for last season'''

#Create 'goals delta'
df['goals_delta'] = df['goals'] - df['goals_LY']

#Create 'last_year_club' and 'last_year_league'
df[['last_year_club','last_year_league']] = ''
for i in range(len(df)):
    if df.iloc[i]['year'] != '2014':
        #get iloc for the relevant columns
        year = df.loc[i]['year']
        id = df.loc[i]['id']
        ly_club_idx, ly_league_idx = df.columns.get_loc('last_year_club'), df.columns.get_loc('last_year_league')
        try:
            df.iloc[i,ly_club_idx] = df.loc[(df['id'] == id) & (df['year'] == last_year_dictionary[year])]['team_title']
            df.iloc[i,ly_league_idx] = df.loc[(df['id'] == id) & (df['year'] == last_year_dictionary[year])]['league']
        except:
            continue

In [18]:
#Now let's create a binary yes/no for whether or not a player changed league/team
df['changed_club'] = (df['team_title'] != df['last_year_club']) & (df['last_year_club'] != '')
df['changed_league'] = (df['league'] != df['last_year_league']) & (df['last_year_club'] != '')

In [19]:
print('Number of players who changed club')
print(np.sum(df[['changed_league','changed_club']]))

Number of players who changed club
changed_league    1079
changed_club      3183
dtype: int64


Now let's perform an F test on those samples to see if they have an affect on goals delta. 

F Test formula: 
(x-µ)/(sigma/np.sqrt(n))  (we can use sigma because we know the population variance). We're looking for a 90% confidence

In [20]:
#First drop null values
df_not_null = df.dropna()
#Create mean for population
goals_delta_mean = np.mean(df_not_null['goals_delta'])
#Calculate no. of players changing, and those sample means
players_change_club = len(df_not_null[df_not_null['changed_club'] == 1])
players_change_league = len(df_not_null[df_not_null['changed_league'] == 1])
goals_delta_mean_change_club = np.mean(df_not_null[df_not_null['changed_club'] == 1]['goals_delta'])
goals_delta_mean_no_change_club = np.mean(df_not_null[df_not_null['changed_club'] == 0]['goals_delta'])
goals_delta_mean_change_league = np.mean(df_not_null[df_not_null['changed_league'] == 1]['goals_delta'])
goals_delta_mean_no_change_league = np.mean(df_not_null[df_not_null['changed_league'] == 0]['goals_delta'])
#Calculate pop. sigma
sigma = np.sqrt(np.var(df_not_null['goals_delta']))
#Perform F test
Z_val_changed_club = (goals_delta_mean_change_club - goals_delta_mean_no_change_club)/(sigma/np.sqrt(players_change_club-1))
Z_val_changed_league = (goals_delta_mean_change_league - goals_delta_mean_no_change_league)/(sigma/np.sqrt(players_change_league-1))
print(f'Z val for changing club: {Z_val_changed_club}\nZ val for changing league: {Z_val_changed_league}')

Z val for changing club: -0.9629322129874146
Z val for changing league: -1.9680133433694724


We can reject the null hypothesis that changing league has no effect at the 95% confidence level, but we cannot reject the null hypothesis that changing club has no effect.

## Running T Test using scipy package

In [25]:
from scipy.stats import ttest_ind
T_test_clubs = ttest_ind(df_not_null[df_not_null['changed_club'] == 1]['goals_delta'],df_not_null[df_not_null['changed_club'] == 0]['goals_delta'])
T_test_leagues = ttest_ind(df_not_null[df_not_null['changed_league'] == 1]['goals_delta'],df_not_null[df_not_null['changed_league'] == 0]['goals_delta'])
t_test_df = pd.DataFrame({'tStat':[T_test_clubs[0],T_test_leagues[0]],'P_score':[T_test_clubs[1],T_test_leagues[1]]},index=['Changed club','Changed league'])

In [26]:
t_test_df

,tStat,P_score
Changed club,-0.810238,0.417821
Changed league,-1.868233,0.061756


Why do we have different results? Z Stat uses population variation, and the T test assumes a wider distribution, so requires a higher value. But both tell us a similar story, which is that changing league is likely to be a factor, whereas changing club is not.

## Plotting joint distributions to see, by eye, which of the values seems to have the biggest impact.

In [27]:
#Let's split up non-pen goals and goals into non-pen goals and pen goals
df_not_null['pen_goals_norm'] = df_not_null['goals_norm'] - df_not_null['npg_norm']
df_not_null['pen_goals_LY_norm'] = df_not_null['goals_LY_norm'] - df_not_null['npg_LY_norm']

<ipython-input-27-266d7bf4f009>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-27-266d7bf4f009>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [28]:
df_not_null.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10904 entries, 0 to 18048
Data columns (total 52 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  10904 non-null  object 
 1   player_name         10904 non-null  object 
 2   games               10904 non-null  object 
 3   time                10904 non-null  object 
 4   goals               10904 non-null  object 
 5   xG                  10904 non-null  float64
 6   assists             10904 non-null  object 
 7   xA                  10904 non-null  float64
 8   shots               10904 non-null  object 
 9   key_passes          10904 non-null  object 
 10  yellow_cards        10904 non-null  object 
 11  red_cards           10904 non-null  object 
 12  position            10904 non-null  object 
 13  team_title          10904 non-null  object 
 14  npg                 10904 non-null  object 
 15  npxG                10904 non-null  float64
 16  xGCh

In [29]:
# x_cols_continuous = ['goals_LY_norm', 'npg_LY_norm', 'xG_LY_norm','npxG_LY_norm', 'changed_league', 'pen_goals_LY_norm','time_LY_norm']
# fig = plotly.subplots.make_subplots(rows=1,cols=7)
#     scat_1 = go.Scatter(x=df_not_null[col],y=df_not_null['goals_norm'],mode='markers',name=col)
#     fig.add_trace(scat_1, row=1, col=idx+1)

# fig.update_layout(width=1200)

# fig.show()

## We seem to have a positive correlation between all variables except pen goals and changed_leage(neg correlation)

In [30]:
df_not_null['position'].value_counts()

M S        2216
D S        2030
F M S      1661
D M S      1011
F S        1002
D           917
GK          711
S           474
M           259
D M         202
D F M S     175
GK S         92
F M          72
F            57
D F M        13
D F S        12
Name: position, dtype: int64

Looking at position, a lot of players have more than one position and it's not clear the weighting between those positions, so someone might be labelled "M S" despite only playing one game in midfield (M). 

Right now, we'll leave this and export the file to run linear regression in a different notebook

In [33]:
fname = 'csv_for_lin_reg_scoring'
df_not_null.to_csv(fname)

In [32]:
df_not_null.describe()

,xG,xA,npxG,xGChain,xGBuildup,goals_LY,npg_LY,xG_LY,npxG_LY,time_LY,...,xA_norm,npxG_norm,xGChain_norm,xGBuildup_norm,goals_LY_norm,npg_LY_norm,xG_LY_norm,npxG_LY_norm,time_LY_norm,pen_goals_LY_norm
count,10904.000000,10904.000000,10904.000000,10904.000000,10904.000000,10904.000000,10904.000000,10904.000000,10904.000000,10904.000000,...,10904.000000,10904.000000,10904.000000,10904.000000,10904.000000,10904.000000,10904.000000,10904.000000,10904.000000,10904.000000
mean,2.143678,1.513903,1.947058,5.783926,3.454150,2.363811,2.158291,2.358712,2.157829,1638.056152,...,1.560181,2.005595,5.961363,3.559580,2.438971,2.227260,2.434525,2.227489,1690.271042,0.211711
std,3.433211,1.960855,3.001099,5.575523,3.316711,3.945001,3.506376,3.582999,3.152195,941.824463,...,2.020718,3.087588,5.746386,3.420722,4.067887,3.619110,3.700802,3.258836,978.251881,0.831969
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,0.164527,0.134454,0.162092,1.734829,1.008050,0.000000,0.000000,0.225750,0.224474,855.000000,...,0.136516,0.167229,1.783604,1.045199,0.000000,0.000000,0.233461,0.230634,888.397059,0.000000
50%,0.879444,0.775170,0.851417,4.257614,2.544880,1.000000,1.000000,1.048398,1.018927,1668.000000,...,0.803696,0.883308,4.398804,2.641376,1.000000,1.000000,1.075336,1.048406,1714.000000,0.000000
75%,2.580601,2.140099,2.413069,8.112318,4.894326,3.000000,3.000000,2.889429,2.699515,2416.000000,...,2.214604,2.496847,8.391962,5.055036,3.000000,3.000000,2.991152,2.807816,2491.235294,0.000000
max,36.866622,20.620707,32.117727,52.409436,28.058852,48.000000,38.000000,39.308762,32.117725,3420.000000,...,20.620707,33.654919,52.409436,31.359893,48.000000,43.428571,40.518048,38.454934,4641.428571,14.000000
